In [1]:
import matplotlib
matplotlib.use('Agg')
import scipy
from scipy import ndimage
import torch, cv2
import numpy as np
import numpy.ma as ma
import sys
import pdb
import torch

from torch.autograd import Variable
import torchvision.models as models
import torch.nn.functional as F
from torch.utils import data
# from dataset import get_segmentation_dataset
# from network import get_segmentation_model
# from config import Parameters
from collections import OrderedDict
import os
import scipy.ndimage as nd
from math import ceil
from PIL import Image as PILImage

import matplotlib.pyplot as plt
import torch.nn as nn

In [2]:
def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """

    palette = [0] * (num_cls * 3)
    palette[0:3] = (128, 64, 128)       # 0: 'road' 
    palette[3:6] = (244, 35,232)        # 1 'sidewalk'
    palette[6:9] = (70, 70, 70)         # 2''building'
    palette[9:12] = (102,102,156)       # 3 wall
    palette[12:15] =  (190,153,153)     # 4 fence
    palette[15:18] = (153,153,153)      # 5 pole
    palette[18:21] = (250,170, 30)      # 6 'traffic light'
    palette[21:24] = (220,220, 0)       # 7 'traffic sign'
    palette[24:27] = (107,142, 35)      # 8 'vegetation'
    palette[27:30] = (152,251,152)      # 9 'terrain'
    palette[30:33] = ( 70,130,180)      # 10 sky
    palette[33:36] = (220, 20, 60)      # 11 person
    palette[36:39] = (255, 0, 0)        # 12 rider
    palette[39:42] = (0, 0, 142)        # 13 car
    palette[42:45] = (0, 0, 70)         # 14 truck
    palette[45:48] = (0, 60,100)        # 15 bus
    palette[48:51] = (0, 80,100)        # 16 train
    palette[51:54] = (0, 0,230)         # 17 'motorcycle'
    palette[54:57] = (119, 11, 32)      # 18 'bicycle'
    palette[57:60] = (105, 105, 105)
    return palette

def pad_image(img, target_size):
    """Pad an image up to the target size."""
    rows_missing = target_size[0] - img.shape[2]
    cols_missing = target_size[1] - img.shape[3]
    padded_img = np.pad(img, ((0, 0), (0, 0), (0, rows_missing), (0, cols_missing)), 'constant')
    return padded_img
def predict_whole_img_w_label(net, image, classes, method, scale, label):
    """
         Predict the whole image w/o using multiple crops.
         The scale specify whether rescale the input image before predicting the results.
    """
#     N_, C_, H_, W_ = image.shape
#     if torch_ver == '0.4':
#         interp = nn.Upsample(size=(H_, W_), mode='bilinear', align_corners=True)
#     else:
#     interp = nn.Upsample(size=(H_, W_), mode='bilinear')

#     bug
#     if scale > 1:
#         scaled_img = ndimage.zoom(image, (1.0, 1.0, scale, scale), order=1, prefilter=False)
#     else:
#         scaled_img = image

#     scaled_img = ndimage.zoom(image, (1.0, 1.0, scale, scale), order=1, prefilter=False)
    
    full_prediction_= net(Variable(torch.from_numpy(image), volatile=True).cuda())
#     full_prediction_ = net('STDCNet1446',19)
    if 'dsn' in method or 'center' in method or 'fuse' in method:
        full_prediction = full_prediction_[-1]
    else:
        full_prediction = full_prediction_[0]

    full_prediction = F.upsample(input=full_prediction, size=(1024, 2048), mode='bilinear', align_corners=True)
    result = full_prediction.cpu().data.numpy().transpose(0,2,3,1)
    return result
def get_confusion_matrix(gt_label, pred_label, class_num):
        """
        Calcute the confusion matrix by given label and pred
        :param gt_label: the ground truth label
        :param pred_label: the pred label
        :param class_num: the nunber of class
        :return: the confusion matrix
        """
        index = (gt_label * class_num + pred_label).astype('int32')
        label_count = np.bincount(index)
        confusion_matrix = np.zeros((class_num, class_num))

        for i_label in range(class_num):
            for i_pred_label in range(class_num):
                cur_index = i_label * class_num + i_pred_label
                if cur_index < len(label_count):
                    confusion_matrix[i_label, i_pred_label] = label_count[cur_index]

        return confusion_matrix
def visulize_attention_ratio(img_path, attention_mask, ratio=0.5, cmap="jet"):
    """
    img_path: 读取图片的位置
    attention_mask: 2-D 的numpy矩阵
    ratio:  放大或缩小图片的比例，可选
    cmap:   attention map的style，可选
    """
    print("load image from: ", img_path)
    # load the image
    img = PILImage.open(img_path, mode='r')
    img_h, img_w = img.size[0], img.size[1]
    plt.subplots(nrows=1, ncols=1, figsize=(0.02 * img_h, 0.02 * img_w))
#     print(img)

    # scale the image
    img_h, img_w = int(img.size[0] * ratio), int(img.size[1] * ratio)
    img = img.resize((img_h, img_w))
    plt.imshow(img, alpha=1)
    plt.axis('off')
    
    # normalize the attention mask
    print(attention_mask)
    mask = cv2.resize(attention_mask, (img_h, img_w))
    normed_mask = mask / mask.max()
    normed_mask = (normed_mask * 255).astype('uint8')
    plt.imshow(normed_mask, alpha=0.5, interpolation='nearest', cmap=cmap)
    plt.savefig("heatmap_nodetail.png")
    plt.show()
    

In [3]:
from models.model_stages import BiSeNet
from cityscapes import CityScapes
%matplotlib inline
def main():
    """Create the model and start the evaluation process."""
#     args = Parameters().parse()

    # file_log = open(args.log_file, "w")
    # sys.stdout = sys.stderr = file_log

#     print("Input arguments:")
#     sys.stdout.flush()
#     for key, val in vars(args).items():
#         print("{:16} {}".format(key, val))

    h, w = map(int, [768,1536])
    input_size = (h, w)

    output_path = './my_predict_train'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    CRnet = BiSeNet('STDCNet1446', 19,use_boundary_2=False, use_boundary_4=False, 
     use_boundary_8=True, use_boundary_16=False, 
     use_conv_last=False)

    ignore_label = 255
    id_to_trainid = {-1: ignore_label, 0: ignore_label, 1: ignore_label, 2: ignore_label,
          3: ignore_label, 4: ignore_label, 5: ignore_label, 6: ignore_label,
          7: 0, 8: 1, 9: ignore_label, 10: ignore_label, 11: 2, 12: 3, 13: 4,
          14: ignore_label, 15: ignore_label, 16: ignore_label, 17: 5,
          18: ignore_label, 19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12, 26: 13, 27: 14,
          28: 15, 29: ignore_label, 30: ignore_label, 31: 16, 32: 17, 33: 18}


    os.environ["CUDA_VISIBLE_DEVICES"]='0'
    methodName = 'train_STDC2-Seg_depthwise51/pths'
    print('loading parameters...')
    respth = '/xiaoou/STDC-Seg-master/STDC-Seg-master/checkpoint/{}/'.format(methodName)
    save_pth = os.path.join(respth, 'model_maxmIOU{}.pth'.format(75))
    store_output = 'True'
    
    saved_state_dict = torch.load(save_pth)
    CRnet.load_state_dict(saved_state_dict,False)

    model = nn.DataParallel(CRnet)
    model.eval()
    model.cuda()
#     dspth = '/xiaoou/my_last_project/dataset/cityscapes'
    dspth = '/xiaoou/STDC-Seg-master/STDC-Seg-master/cityscapes'
    randomscale = (0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375, 1.5)
    dsval = CityScapes(dspth,cropsize=[1024,2048],mode='val',randomscale=randomscale)
    testloader = data.DataLoader(dsval,
                    batch_size = 2,
                    shuffle = False,
                    num_workers = 2,
                    drop_last = False)

    data_list = []
    confusion_matrix = np.zeros((19,19))

    palette = get_palette(20)

    image_id = 0
    for index, batch in enumerate(testloader):
        if index % 100 == 0:
            print('%d processd'%(index))
            sys.stdout.flush()
        image, label,name = batch
        size = image.shape
#         size = size[0]
        with torch.no_grad():
            if 'val' in 'val':
                output= predict_whole_img_w_label(model, image.numpy(), 19, 
                            'val', scale=float(1), label=Variable(label.long().cuda()))
                
            else:
                output = predict_whole_img(model, image.numpy(),19, 
                    'val', scale=float(1))
    
#     output = np.squeeze(output)
#     print(output.shape)
#     visulize_attention_ratio('/xiaoou/STDC-Seg-master/STDC-Seg-master/cityscapes/leftImg8bit/val/aachen/aachen_000096_000019_leftImg8bit.png',output)
#         print(output.shape)
        seg_pred = np.asarray(np.argmax(output, axis=3), dtype=np.uint8)
        m_seg_pred = ma.masked_array(seg_pred, mask=torch.eq(label, 255))
        ma.set_fill_value(m_seg_pred, 20)
        seg_pred = m_seg_pred
#         seg_pred = np.expand_dims(seg_pred, 1)
#         print(seg_pred.shape)

        for i in range(image.size(0)): 
            image_id += 1
            print('%d th segmentation map generated ...'%(image_id))
            sys.stdout.flush()
            if store_output == 'True':
                output_im = PILImage.fromarray(seg_pred[i])
                output_im.putpalette(palette)
                output_im.save(output_path+'/'+name[i]+'_gtFine_labelIds.png')
#         print(label.shape)
        seg_pred = np.expand_dims(seg_pred, 1)
        seg_gt = np.asarray(label.numpy())#[:,:size[0],:size[1]], dtype=np.int)
#         print(seg_gt.shape)
        ignore_index = seg_gt != 255
        seg_gt = seg_gt[ignore_index]
        
        seg_pred = seg_pred[ignore_index]
        confusion_matrix += get_confusion_matrix(seg_gt, seg_pred, 19)
            
    pos = confusion_matrix.sum(1)
    res = confusion_matrix.sum(0)
    tp = np.diag(confusion_matrix)

    IU_array = (tp / np.maximum(1.0, pos + res - tp))
    mean_IU = IU_array.mean()
    
    print({'meanIU':mean_IU, 'IU_array':IU_array})

    print("confusion matrix\n")
#     print(confusion_matrix)
    sys.stdout.flush()

main()

loading parameters...


/xiaoou/STDC-Seg-master/STDC-Seg-master/models/model_stages.py:129: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.weight, 0)
/xiaoou/STDC-Seg-master/STDC-Seg-master/models/model_stages.py:130: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.bias, 0)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [6]:
import numpy as np
import cv2
from PIL import Image
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os

def visulize_attention_ratio(img_path, attention_mask, ratio=0.5, cmap="jet"):
    """
    img_path: 读取图片的位置
    attention_mask: 2-D 的numpy矩阵
    ratio:  放大或缩小图片的比例，可选
    cmap:   attention map的style，可选
    """
    print("load image from: ", img_path)
    # load the image
    img = Image.open(img_path, mode='r')
    img_h, img_w = img.size[0], img.size[1]
    plt.subplots(nrows=1, ncols=1, figsize=(0.02 * img_h, 0.02 * img_w))

    # scale the image
    img_h, img_w = int(img.size[0] * ratio), int(img.size[1] * ratio)
    img = img.resize((img_h, img_w))
    plt.imshow(img, alpha=1)
    plt.axis('off')
    
    # normalize the attention mask
    mask = cv2.resize(attention_mask, (img_h, img_w))
    normed_mask = mask / mask.max()
    normed_mask = (normed_mask * 255).astype('uint8')
    plt.imshow(normed_mask, alpha=0.5, interpolation='nearest', cmap=cmap)
visulize_attention_ratio('/xiaoou/STDC-Seg-master/STDC-Seg-master/cityscapes/leftImg8bit/val/aachen/aachen_000096_000019_leftImg8bit.png',output,)

NameError: name 'output' is not defined

In [1]:
##+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## Created by: RainbowSecret
## Microsoft Research
## yuyua@microsoft.com
## Copyright (c) 2018
##
## This source code is licensed under the MIT-style license found in the
## LICENSE file in the root directory of this source tree 
##+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import argparse
import scipy
from scipy import ndimage
import torch, cv2
import numpy as np
import sys
import pdb

from torch.autograd import Variable
import torchvision.models as models
import torch.nn.functional as F
from torch.utils import data
from collections import OrderedDict
# from dataset import get_segmentation_dataset
# from network import get_segmentation_model
# from config import Parameters
import os
import scipy.ndimage as nd
from math import ceil
from PIL import Image as PILImage

import matplotlib.pyplot as plt
import torch.nn as nn
# torch_ver = torch.__version__[:3]
from models.model_stages import BiSeNet
from cityscapes import CityScapes


def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """

    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


def pad_image(img, target_size):
    """Pad an image up to the target size."""
    rows_missing = target_size[0] - img.shape[2]
    cols_missing = target_size[1] - img.shape[3]
    padded_img = np.pad(img, ((0, 0), (0, 0), (0, rows_missing), (0, cols_missing)), 'constant')
    return padded_img


def predict_sliding(net, image, tile_size, classes, method, scale=1):
    if scale != 1:
        scaled_img = ndimage.zoom(image, (1.0, 1.0, scale, scale), order=1, prefilter=False)
    else:
        scaled_img = image

    N_, C_, H_, W_ = scaled_img.shape

    if torch_ver == '0.4':
        interp = nn.Upsample(size=tile_size, mode='bilinear', align_corners=True)
    else:
        interp = nn.Upsample(size=tile_size, mode='bilinear')

    full_probs = np.zeros((N_, H_, W_, classes))
    count_predictions = np.zeros((N_, H_, W_, classes))
    overlap = 0
    stride_h = ceil(tile_size[0] * (1 - overlap))
    stride_w = ceil(tile_size[1] * (1 - overlap))
    tile_rows = int(ceil((H_ - tile_size[0]) / stride_h) + 1)  # strided convolution formula
    tile_cols = int(ceil((W_ - tile_size[1]) / stride_w) + 1)
    print("Need %i x %i prediction tiles @ stride %i px, %i py" % (tile_cols, tile_rows, stride_h, stride_w))

    tile_counter = 0

    for row in range(tile_rows):
        for col in range(tile_cols):
            x1 = int(col * stride_w)
            y1 = int(row * stride_h)
            x2 = min(x1 + tile_size[1], W_)
            y2 = min(y1 + tile_size[0], H_)
            x1 = max(int(x2 - tile_size[1]), 0)  # for portrait images the x1 underflows sometimes
            y1 = max(int(y2 - tile_size[0]), 0)  # for very few rows y1 underflows

            img = scaled_img[:, :, y1:y2, x1:x2]
            padded_img = pad_image(img, tile_size)
            tile_counter += 1
            print("Predicting tile %i" % tile_counter)
            padded_prediction_ = net(Variable(torch.from_numpy(padded_img), volatile=True).cuda(), )
    
            if 'dsn' in method or 'center' in method or 'fuse' in method:
                padded_prediction = padded_prediction_[-1]
            else:
                padded_prediction = padded_prediction_

            padded_prediction = F.upsample(input=padded_prediction, size=tile_size, mode='bilinear', align_corners=True)
            padded_prediction = padded_prediction.cpu().data.numpy().transpose(0,2,3,1)                
            # padded_prediction = interp(padded_prediction).cpu().data.numpy().transpose(0,2,3,1)
            prediction = padded_prediction[:, 0:img.shape[2], 0:img.shape[3], :]
            count_predictions[:, y1:y2, x1:x2] += 1
            full_probs[:, y1:y2, x1:x2] += prediction 

    full_probs /= count_predictions
    full_probs = ndimage.zoom(full_probs, (1., 1./scale, 1./scale, 1.),
        order=1, prefilter=False)
    return full_probs


def predict_whole_img(net, image, classes, method, scale):
    """
         Predict the whole image w/o using multiple crops.
         The scale specify whether rescale the input image before predicting the results.
    """
    N_, C_, H_, W_ = image.shape
    torch_ver = 1.3

#     if torch_ver == '0.4':
#         interp = nn.Upsample(size=(H_, W_), mode='bilinear', align_corners=True)
#     else:
#         interp = nn.Upsample(size=(H_, W_), mode='bilinear')

#     bug
#     if scale > 1:
#         scaled_img = ndimage.zoom(image, (1.0, 1.0, scale, scale), order=1, prefilter=False)
#     else:
#         scaled_img = image

#     scaled_img = ndimage.zoom(image, (1.0, 1.0, scale, scale), order=1, prefilter=False)
    
    full_prediction_ = net(Variable(torch.from_numpy(image), volatile=True).cuda())
    if 'dsn' in method:
        full_prediction = full_prediction_[-1]
    else:
        full_prediction = full_prediction_[0]

    if torch_ver == '0.4':
        full_prediction = F.upsample(input=full_prediction, size=(1024, 2048), mode='bilinear', align_corners=True)
    else:
        full_prediction = F.upsample(input=full_prediction, size=(1024, 2048), mode='bilinear')
    
    result = full_prediction.cpu().data.numpy().transpose(0,2,3,1)

    # result = interp(full_prediction).cpu().data.numpy().transpose(0,2,3,1)
    return result


def predict_multi_scale(net, image, scales, classes, flip_evaluation, method):
    """
    Predict an image by looking at it with different scales.
        We choose the "predict_whole_img" for the image with less than the original input size,
        for the input of larger size, we would choose the cropping method to ensure that GPU memory is enough.
    """
    N_, C_, H_, W_ = image.shape
    full_probs = np.zeros((N_, H_, W_, classes))  
    for scale in scales:
        scale = float(scale)
        print("Predicting image scaled by %f" % scale)
        sys.stdout.flush()
        if scale <= 1.0:
            scaled_probs = predict_whole_img(net, image, classes, method, scale=scale)
        else:        
            scaled_probs = predict_sliding(net, image, (1024,2048), classes, method, scale=scale)
        if flip_evaluation == 'True':
            if scale <= 1.0:
                flip_scaled_probs = predict_whole_img(net, image[:,:,:,::-1].copy(), classes, method, scale=scale)
            else:
                flip_scaled_probs = predict_sliding(net, image[:,:,:,::-1].copy(), (1024,2048), classes, method, scale=scale)
            scaled_probs = 0.5 * (scaled_probs + flip_scaled_probs[:,:,::-1])
        full_probs += scaled_probs
    full_probs /= len(scales)
    return full_probs


def id2trainId(label, id_to_trainid, reverse=False):
        label_copy = label.copy()
        if reverse:
            for v, k in id_to_trainid.items():
                label_copy[label == k] = v
        else:
            for k, v in id_to_trainid.items():
                label_copy[label == k] = v
        return label_copy


def main():
    """Create the model and start the evaluation process."""
#     args = Parameters().parse()

    # file_log = open(args.log_file, "w")
    # sys.stdout = sys.stderr = file_log

#     print("Input arguments:")
#     for key, val in vars(args).items():
#         print("{:16} {}".format(key, val))
#     sys.stdout.flush()

    h, w = map(int, [768,1536])
    input_size = (h, w)
    ignore_label= 255
    output_path = '/xiaoou/STDC-Seg-master/STDC-Seg-master/predict_submit_74.7'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    CRnet = BiSeNet('STDCNet1446', 19,use_boundary_2=False, use_boundary_4=False, 
     use_boundary_8=True, use_boundary_16=False, 
     use_conv_last=False)

    os.environ["CUDA_VISIBLE_DEVICES"]='0'
    
    methodName = 'train_STDC2-Seg_depthwise60/pths'
    print('loading parameters...')
    respth = '/xiaoou/STDC-Seg-master/STDC-Seg-master/checkpoint/{}/'.format(methodName)
    save_pth = os.path.join(respth, 'model_maxmIOU{}.pth'.format(75))
    store_output = 'True'
    saved_state_dict = torch.load(save_pth)
    CRnet.load_state_dict(saved_state_dict,False)

    model = nn.DataParallel(CRnet)
    model.eval()
    model.cuda()
    dspth = '/xiaoou/my_last_project/dataset/cityscapes'
    randomscale = (0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375, 1.5)
    dsval = CityScapes(dspth,cropsize=[1024,2048],mode='test',randomscale=randomscale)
    testloader = data.DataLoader(dsval,
                    batch_size = 2,
                    shuffle = False,
                    num_workers = 2,
                    drop_last = False)

    data_list = []
    confusion_matrix = np.zeros((19,19))
    palette = get_palette(256)

    id_to_trainid = {-1: ignore_label, 0: ignore_label, 1: ignore_label, 2: ignore_label,
                  3: ignore_label, 4: ignore_label, 5: ignore_label, 6: ignore_label,
                  7: 0, 8: 1, 9: ignore_label, 10: ignore_label, 11: 2, 12: 3, 13: 4,
                  14: ignore_label, 15: ignore_label, 16: ignore_label, 17: 5,
                  18: ignore_label, 19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12, 26: 13, 27: 14,
                  28: 15, 29: ignore_label, 30: ignore_label, 31: 16, 32: 17, 33: 18}
    image_id = 0
    for index, batch in enumerate(testloader):
        if index % 100 == 0:
            print('%d processd'%(index))
        image, name = batch
        size = image.shape
        with torch.no_grad():
            if 'val' in 'test': 
                output = predict_multi_scale(model, image.numpy(), ([0.75, 1, 1.25]), 
                    args.num_classes, args.use_flip, args.method)
            else:
                output = predict_whole_img(model, image.numpy(),19, 
                    'test', scale=float(1))


        seg_pred = np.asarray(np.argmax(output, axis=3), dtype=np.uint8)
        seg_pred = id2trainId(seg_pred, id_to_trainid, reverse=True)

        for i in range(image.size(0)): 
            image_id += 1
            print('%d th segmentation map generated ...'%(image_id))
            sys.stdout.flush()
            if store_output == 'True':
                output_im = PILImage.fromarray(seg_pred[i])
                output_im.putpalette(palette)
                output_im.save(output_path+'/'+name[i]+'_gtFine_labelIds.png')

if __name__ == '__main__':
    main()


/xiaoou/STDC-Seg-master/STDC-Seg-master/models/model_stages.py:129: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.weight, 0)
/xiaoou/STDC-Seg-master/STDC-Seg-master/models/model_stages.py:130: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.bias, 0)


loading parameters...
self.mode test
self.len test 1525
0 processd


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:149: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


1 th segmentation map generated ...
2 th segmentation map generated ...
3 th segmentation map generated ...
4 th segmentation map generated ...
5 th segmentation map generated ...
6 th segmentation map generated ...
7 th segmentation map generated ...
8 th segmentation map generated ...
9 th segmentation map generated ...
10 th segmentation map generated ...
11 th segmentation map generated ...
12 th segmentation map generated ...
13 th segmentation map generated ...
14 th segmentation map generated ...
15 th segmentation map generated ...
16 th segmentation map generated ...
17 th segmentation map generated ...
18 th segmentation map generated ...
19 th segmentation map generated ...
20 th segmentation map generated ...
21 th segmentation map generated ...
22 th segmentation map generated ...
23 th segmentation map generated ...
24 th segmentation map generated ...
25 th segmentation map generated ...
26 th segmentation map generated ...
27 th segmentation map generated ...
28 th segm